In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

### This is a work in progress

This is my 3rd. notebook of learning Time Series ML with this Store Sales dataset. The 3rd. attempt here will be focusing on 3 things:
1. Feature Selection
2. More Compilated ML models
3. Pipeline


For anyone interested in the earlier notebooks in the series, here's the link to them.
* [1# attempt : Focusing on EDA](https://www.kaggle.com/code/thanakr/beginner-s-first-project-store-sales)
* [2# attempt : Focusing on data formatting and expanding windows](https://www.kaggle.com/code/thanakr/beginner-s-project-2nd-attempt)

Also I'm trying to improve the score in each attempts and eventually beat the Kaggle's course score of 0.5109. Here's my record.

### Score Log

#### ---- First Notebook----
#### 1st. edit -> RSMLE 3.44178
* LinearRegression
* without checking collinearity
* no zero sales formatting

#### ---- Second Notebook----
#### 2nd. edit -> RSMLE 2.37238
* LinearRegression
* Flawed data formatting
* Need to drop NA
* Wrong formatted data lag
* Collinearity Checked
* Manually mark the zero sales product

#### 3rd. edit -> RSMLE 1.99939 The score Improve Significantly
* LinearRegression
* Drop NA / Lag fixed
* Reduced sales / transactions features to only MA15
* Collinearity Checked
* Manually mark the zero sales product

#### 4th. edit -> RSMLE 1.97205 Best Score Yet
* LinearRegression + Expanding windows
* Drop NA / Lag fixed
* Reduced sales / transactions features to only MA15
* Collinearity Checked
* Manually mark the zero sales product

#### ---- Third Notebook----
we are here


### Let's begin

Reference: 
https://machinelearningmastery.com/modeling-pipeline-optimization-with-scikit-learn/
https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.Pipeline.html


# The Plan
1. Impliment the Feature selection/dimensions reduction method and comparing them.
    * using sklearn.feature_selection
    * using sklearn.decomposition
    
2. Try different models
    * Linear Regression
    * Poisson Regression
    * Polymomial Regression
    * Random Forest
    * XGBoost
    * LGB
    
3. Try build the Pipeline

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

from warnings import simplefilter
simplefilter("ignore")  # ignore warnings to clean up output cells

import gc # for garbage cleaning

%config Completer.use_jedi = False # for autocomplete 

In [ ]:
holiday_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv",)
train_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/train.csv",)
test_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/test.csv")
transaction_data = pd.read_csv("/kaggle/input/store-sales-time-series-forecasting/transactions.csv")

In [ ]:
train_data.tail()

In [ ]:
transaction_data.tail()

In [ ]:
holiday_data.tail()

In [ ]:
test_data.tail()

Constructing the main inputs DataFrame (Code are from the 2nd attempt [notebook.](https://www.kaggle.com/code/thanakr/beginner-s-project-2nd-attempt/notebook))

In [ ]:
stack_sales = pd.concat([train_data,test_data]) #stack train and test together for them to be formatted together.

stack_sales['date'] = pd.to_datetime(stack_sales['date']) #format 'date' column as datetime

transaction_data['date'] = pd.to_datetime(transaction_data['date']) #format 'date' column as datetime

stack_sales = stack_sales.merge(transaction_data,how='left',on=['date','store_nbr']) # merge then together

stack_sales = stack_sales.sort_values(['family','store_nbr','date']).reset_index() # sort by product family and store for picking the right lag

stack_sales.fillna(0, inplace= True) # Fill sales and transcation NaN with zero

#----this section is for manually making the dummy variabie ----
nbr_dummy = pd.get_dummies(stack_sales['store_nbr'],prefix='store',drop_first=True) # create dummy
family_dummy = pd.get_dummies(stack_sales['family'],prefix='fam',drop_first=True) # create dummy

stack_sales = stack_sales.merge(nbr_dummy, how ='left', left_index = True, right_index = True) # merge dummy
stack_sales = stack_sales.merge(family_dummy, how ='left', left_index = True, right_index = True) # merge dummy

stack_sales.drop(['store_nbr','family'],axis = 1, inplace = True)
#----this section is for manually making the dummy variabie ----

stack_sales.drop(['index'],axis = 1, inplace = True)

stack_sales.head()

Constructing the holiday section of main inputs DataFrame (Code adapted from the 2nd attempt [notebook.](https://www.kaggle.com/code/thanakr/beginner-s-project-2nd-attempt/notebook))


In [ ]:
holiday = holiday_data.set_index('date')
holiday.index = pd.to_datetime(holiday.index) #format index as datetime
nat_holiday_data = holiday.loc[holiday['locale'].str.contains('National')] #filter only National Holidays

calendar = pd.DataFrame(index = pd.date_range('2013-01-01','2017-08-31')) #create datetime index from first to last day to use as anchors
calendar = calendar.join(nat_holiday_data).fillna(0) #merge holiday with datetime anchor

calendar['is_ny'] = 0 # add new year dummy
calendar.loc['2013-01-01', 'is_ny'] = 1 # add new year 
calendar.loc['2014-01-01', 'is_ny'] = 1 # add new year 
calendar.loc['2015-01-01', 'is_ny'] = 1 # add new year 
calendar.loc['2016-01-01', 'is_ny'] = 1 # add new year 
calendar.loc['2017-01-01', 'is_ny'] = 1 # add new year 

calendar['dow'] = calendar.index.weekday+1 #set Monday to 1 and Sunday to 7
calendar['moy'] = calendar.index.month 

calendar['is_non_workday'] = 0

calendar.loc[calendar['locale'] == 'National', 'is_non_workday'] = 1 #make national events a non_workday
calendar.loc[calendar['dow'] > 5 , 'is_non_workday'] = 1 #make Sat and Sun non_workday
calendar.loc[calendar['type'] == 'Work Day', 'is_non_workday'] = 0 #make Work Day holiday a working day
calendar.loc[(calendar['transferred'] == True) & 
             (calendar['dow'] < 6), 
             'is_non_workday'] = 0 #mark a transfered weekday a working day

calendar['is_football'] = 0 #add football dummy column
calendar['is_eq'] = 0 #add earthquake dummy column
calendar['is_shopping'] = 0 # add shopping events dummy column

#mark football matches
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('futbol')), 'is_football'] = 1

#mark the earthquake
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('Terremoto')), 'is_eq'] = 1

#mark Cyber Mondays
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('Cyber Monday')), 'is_shopping'] = 1

#mark Black Fridays
calendar.loc[(calendar['locale'] == 'National') & 
             (calendar['description'].str.contains('Black Friday')), 'is_shopping'] = 1

#----this section is for manually making the dummy variabie ----

dow_dummy = pd.get_dummies(calendar['dow'],prefix = 'dow',drop_first = True) #make dummy for day of week
moy_dummy = pd.get_dummies(calendar['moy'],prefix = 'moy',drop_first = True) # make dummy for month of yeawr

calendar = calendar.merge(dow_dummy, how='left', left_index = True, right_index = True) #merge dow dummy
calendar = calendar.merge(moy_dummy, how='left', left_index = True, right_index = True) #merge moy dummy

#slice only the needed columns

calendar_rdy = calendar[['is_ny',
       'is_non_workday', 'is_football', 'is_eq', 'is_shopping',
       'moy_2', 'moy_3',
       'moy_4', 'moy_5', 'moy_6', 'moy_7', 'moy_8', 'moy_9', 'moy_10',
       'moy_11', 'moy_12']] 

#----this section is for manually making the dummy variabie ----


#calendar_rdy = calendar[['is_ny','is_non_workday', 'is_football', 'is_eq', 'is_shopping','dow','moy']] 

In [ ]:
calendar_rdy.tail()

In [ ]:
stack_sales = stack_sales.merge(calendar_rdy, how ='left', left_on = 'date', right_on = calendar_rdy.index) #merge sales with holiday

In [ ]:
del calendar
del calendar_rdy
del holiday_data
del train_data
del moy_dummy
del dow_dummy

gc.collect()

Let's try various feature engineering. They will be filtered with feature selection.

In [ ]:
from sklearn.preprocessing import StandardScaler

def sd_scaling(df, column_names):
    sdScale = StandardScaler()
    for col in column_names:
        sdScale.fit(df[[col]])
        df['{}_scaled'.format(col)] = sdScale.transform(df[[col]]) 
        
sd_scaling(stack_sales, ['sales','onpromotion', 'transactions'])

In [ ]:
# def make_column_lag(df, column_name, lag_list):
#     for lag in lag_list:
#         df['{}_lag_{}'.format(column_name, lag)] = df[column_name].shift(lag)
        
# make_column_lag(stack_sales,'sales_scaled',[1 ,2 ,3 ,4 ,5 ,6 ,7 ,8 ,13 ,14])
# make_column_lag(stack_sales,'transactions_scaled',[21, 22, 28])

In [ ]:
def make_column_ma(df, column_name, ma_list):
    for ma in ma_list:
        df['{}_ma_{}'.format(column_name, ma)] = df[column_name].rolling(ma, 
                                                                         min_periods = ma, 
                                                                         closed='left').mean()
#ma_list = [8, 15, 31]
ma_list = [8]
# ma_list+1 is to shift the lag further back 1 step to offset the exclusion of the current day.
make_column_ma(stack_sales,'sales_scaled',ma_list)
make_column_ma(stack_sales,'transactions_scaled',ma_list)

In [ ]:
stack_sales = stack_sales.drop(['onpromotion','transactions','sales_scaled','transactions_scaled'],axis = 1)
stack_sales.dropna(inplace=True)

check the current columns.

In [ ]:
X_train = stack_sales.set_index('date').loc[:'2017-08-15'].drop(['id','sales'],axis=1)
y_train = stack_sales.set_index('date').loc[:'2017-08-15', 'sales']
X_test = stack_sales.set_index('date').loc['2017-08-16':].drop(['id','sales'],axis=1)

In [ ]:
del stack_sales
gc.collect()

# Comparing Models

Here I will write a loop to test all the model listed above with each featured selection criteria tested earlier. They will be compared with the accuracy to see how well thay can produce the output with the reduced feature.

### Let't begin by train and predict model using the full inputs as a benchmark.

* LinearRegression
* ~~PolynomialFeature~~ Computational expensive for PolynomialFeature()
    * will be test again in feature selection
* Poisson Distribution Regression
* ~~Randomforest~~ 
    * using RandomForest with timeseries regression is not a very good choice. Random Forest can't extrapolate for the newly introduced magnitude of features
* XGBoost
* LGB


In [ ]:
from sklearn.linear_model import LinearRegression, PoissonRegressor

from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
X_train.info(verbose=True)

In [ ]:
# from sklearn.metrics import mean_squared_log_error
from sklearn.pipeline import make_pipeline 
from sklearn.preprocessing import PolynomialFeatures

import re
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  #fix the "LightGBMError: Do not support special JSON characters in feature name."
X_train.tail()

lr_pipe = make_pipeline(LinearRegression())
# poly_pipe = make_pipeline(PolynomialFeatures(degree=3, include_bias=False),LinearRegression())
poisson_pipe = make_pipeline(PoissonRegressor())
xgb_pipe = make_pipeline(XGBRegressor(tree_method='hist'))
lgb_pipe = make_pipeline(LGBMRegressor())

pipes = [lr_pipe,poisson_pipe,xgb_pipe,lgb_pipe]
pipe_dic = {0: 'Linear Regression',1:'Poission Regression',2:'XGBregressor',3:'LGBRegressor'}

From all available features (all the dummy and only lag/ma 8) Xgb seem to do the best job at accuracy on the training job. Let's see them working with the test data.

In [ ]:
y_pred_dic = {}

for i,pipe in enumerate(pipe_dic):
    print('training {}'.format(pipe_dic[i]))
    pipes[i].fit(X_train,y_train)
    print('{} score : {} \n'.format(pipe_dic[i],pipes[i].score(X_train,y_train)))
    print('Predict using {} \n'.format(pipe_dic[i]))
    y_pred_dic[pipe] = pipes[i].predict(X_test)
    
    

In [ ]:
y_pred_all_feature_df = pd.DataFrame(y_pred_dic)
y_pred_all_feature_df = y_pred_all_feature_df.rename(pipe_dic,axis=1)
y_pred_all_feature_df

In [ ]:
y_pred_all_feature_df = test_data.merge(y_pred_all_feature_df,how='left',left_index=True, right_index=True)

let's compare them together using line plot

In [ ]:

fig,axs = plt.subplots(4,figsize=(30,30))


axs[0].plot(y_pred_all_feature_df['Linear Regression'],c='r',label='Linear Regression')
axs[1].plot(y_pred_all_feature_df['Poission Regression'],c='g',label='Poission Regression')
axs[2].plot(y_pred_all_feature_df['XGBregressor'],c='b',label='XGBregressor')
axs[3].plot(y_pred_all_feature_df['LGBRegressor'],c='purple',label='LGBRegressor')

plt.show()

let's try submit the result from all feature just to see the score. I will take a note of the test score below


In [ ]:
y_pred_all_feature_df

In [ ]:
submit = y_pred_all_feature_df[['id','LGBRegressor']]
submit = submit.rename({'LGBRegressor':'sales'}, axis=1)
submit.to_csv('submission.csv',index=False)

Linear Regression score : 0.8878299241319337 
Predict using Linear Regression -> Score 1.97205

Poission Regression score : 0.8856629555251515 
Predict using Poission Regression -> Score 2.86212

XGBregressor score : 0.9468573006502172 
Predict using XGBregressor -> Score 3.57705

LGBRegressor score : 0.9340568917907807 
Predict using LGBRegressor -> Score 

In [ ]:
#current working cell

# Feature Selection

Now, let's apply some feature selection class.
* VarianceThreshold -> Drop thge low varience feature.
* SelectKbest -> picking only the big weight features
* ~~Recursive feature elimination -> Recursively pick the good features in or filter bad features out.~~ Computationally Expensive
* Seelct From model -> pick only the bigger weight from the .coef_ attribute after the model is fitted.

In [ ]:
from sklearn.feature_selection import VarianceThreshold, SelectFromModel, SelectKBest, f_regression

In [ ]:
#VarianceThreshold
var_thresh = VarianceThreshold()
var_thresh_X_train = var_thresh.fit(X_train)

In [ ]:
var_thresh_X_train = var_thresh_X_train.transform(X_train)

In [ ]:
print(var_thresh_X_train.shape)
print(X_train.shape)

In [ ]:
del var_thresh
del var_thresh_X_train
gc.collect()

Seem like the varianceThreshold can't reduce any dimension here. let's continue on SelectKBest

In [ ]:
k_best = SelectKBest(f_regression, k=100) #let's keep only 100 features
k_best.fit(X_train,y_train)

In [ ]:
col = k_best.get_support(indices=True)
X_train.iloc[:,col].columns

In [ ]:
k_best = k_best.transform(X_train)

In [ ]:
print(k_best.shape)
print(X_train.shape)

Now let's do the SelecrFromModel Class.

In [ ]:
#linear regression
from_model_lr = SelectFromModel(LinearRegression(fit_intercept=True))
from_model_lr = from_model_lr.fit(X_train,y_train)

In [ ]:
from_model_lr.get_feature_names_out()

This reduce a lot of dimentionality for linear regression. Now, let's try polynomial

In [ ]:
X_train_to_poly = X_train[['onpromotion_scaled','sales_scaled_ma_8','transactions_scaled_ma_8']]

In [ ]:
poly_transformer = PolynomialFeatures(degree=3, include_bias=False)
X_train_to_poly = poly_transformer.fit_transform(X_train_to_poly)

In [ ]:
temp = pd.DataFrame(X_train_to_poly,columns = ['poly_{}'.format(i) for i in range(19)])

In [ ]:
X_train

In [ ]:
temp = temp.merge(X_train.reset_index(),how='left',left_index=True,right_index=True)

In [ ]:
temp = temp.drop(['onpromotion_scaled','sales_scaled_ma_8','transactions_scaled_ma_8'], axis = 1)

In [ ]:
temp.set_index('date', inplace=True)

In [ ]:
temp

In [ ]:
from_model_poly = SelectFromModel(LinearRegression())
from_model_poly = from_model_poly.fit(temp,y_train)

In [ ]:
from_model_poly.get_feature_names_out()

Let's try Poisson Distribution Regressor

In [ ]:
from_model_p = SelectFromModel(PoissonRegressor())
from_model_p = from_model_p.fit(X_train,y_train)

In [ ]:
from_model_p.get_feature_names_out()

feature for XGBoost

In [ ]:
xgb = XGBRegressor(tree_method='hist')
xgb = xgb.fit(X_train,y_train)

Let's see the top 20 most important features for XGB.

In [ ]:
from xgboost import plot_importance
plot_importance(xgb,max_num_features=20,)

Feature for LGB

In [ ]:
import re
X_train = X_train.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))  #fix the "LightGBMError: Do not support special JSON characters in feature name."


lgb = LGBMRegressor()
lgb.fit(X_train,y_train)

In [ ]:
importance = pd.DataFrame()
importance['Features'] = lgb.feature_name_
importance['Importance']  = lgb.feature_importances_

importance.sort_values('Importance',ascending=False)[:21]

lgb weight a bit different than xgb. For instance, is_eq has more say than fam_GROCERYI.

Create formatting pipeline